# Setup

In [ ]:
! pip install --upgrade --force-reinstall git+https://github.com/jinmingteo/cocoapi.git#subdirectory=PythonAPI

  Cloning https://github.com/jinmingteo/cocoapi.git to /tmp/pip-req-build-gby5d_c1
  Running command git clone -q https://github.com/jinmingteo/cocoapi.git /tmp/pip-req-build-gby5d_c1
     |████████████████████████████████| 583kB 4.6MB/s 
     |████████████████████████████████| 2.0MB 43.8MB/s 
     |████████████████████████████████| 12.4MB 18.3MB/s 
     |████████████████████████████████| 92kB 7.5MB/s 
     |████████████████████████████████| 20.1MB 1.3MB/s 
     |████████████████████████████████| 235kB 50.3MB/s 
     |████████████████████████████████| 71kB 9.7MB/s 
  Created wheel for pycocotools: filename=pycocotools-2.0-cp36-cp36m-linux_x86_64.whl size=267054 sha256=a347fb3b5e281a163cc16ce7f3e4bdf4e966a7f19389cf814668b378448e6bb2
  Stored in directory: /tmp/pip-ephem-wheel-cache-yk4plusf/wheels/27/81/92/3a512329d1b1ae7fc278285a1f114ef08082568bf32eee0002
Successfully built pycocotools
ERROR: google-colab 1.0.0 has requirement six~=1.12.0, but you'll have six 1.15.0 which is incompatib

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# add shortcut to WASD++ to your drive before running this
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/WASD++/CV')

In [ ]:
import json
import torch
import yaml
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

# Weighted Box Ensemble

In [ ]:
!pip install ensemble-boxes

In [ ]:
import pandas as pd
from PIL import ImageEnhance, ImageFont, ImageDraw, Image
import numpy as np
from ensemble_boxes import *
import sys
from tqdm import tqdm
import os
import warnings
warnings.filterwarnings("ignore")

## Load Results of models

## For interim

In [ ]:
df1 = pd.read_json("json_outputs/interim/yolov4_interim.json") 
df2 = pd.read_json("json_outputs/interim/efficientdetd4_tf_interim_0.660.json")
df3 = pd.read_json("json_outputs/interim/yolov5_0.637interim.json")
df4 = pd.read_json("json_outputs/interim/efficientdetd4_pytorch_interim.json")
df5 = pd.read_json("json_outputs/interim/retina_combined15epochs_interim.json")

## For val

In [ ]:
df1 = pd.read_json("json_outputs/val/yolov4_0.664.json") 
df2 = pd.read_json("json_outputs/val/efficientdetd4_tf_0.660.json")
df3 = pd.read_json("json_outputs/val/yolov5_0.637val.json")
df4 = pd.read_json("json_outputs/val/efficientdetd4_pytorch_0.626.json")
df5 = pd.read_json("json_outputs/val/retina_0.495.json")

In [ ]:
print(len(df1), len(df2), len(df3), len(df4), len(df5))

4175 90644 25133 23430 91909


## Create Ensemble json

In [ ]:
dataset_name = "interim_val"
weights = [5, 4, 3, 2, 1]
# weights = [4, 3, 2, 3, 1]

In [ ]:
json_data = json.load(open('/content/drive/My Drive/Colab Notebooks/datasets/my_data/{}.json'.format(dataset_name)))
categories = json_data['categories']
if dataset_name =="val":
    annotations = json_data['annotations']
images = json_data['images']

In [ ]:
results = []

def wbf_mult_models(boxes_list, scores_list, labels_list, weights, iou_thr=0.55):

    boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, weights=weights, iou_thr=iou_thr, skip_box_thr=0.0)

    return boxes, scores, labels


for x in tqdm(images):
  path ='/content/drive/My Drive/Colab Notebooks/datasets/my_data/{}/{}/{}'.format(dataset_name,dataset_name,x['file_name'])
  img = Image.open(path)
  w,h = img.size

  #Getting Efficient Det model predictions for this image id
  box1 = df1[df1.image_id == x['id']]
  if box1 is None:
    print("No Predictions")
  # print("Number of predictions from EfficientDet {}".format(str(len(test))))
  box1['bbox_edit'] =  np.empty((len(box1), 0)).tolist()
  for idx, row in box1.iterrows():
    box1['bbox_edit'][idx] =  [row['bbox'][0]/w, 
                              row['bbox'][1]/h, 
                              (row['bbox'][0] + row['bbox'][2])/w, 
                              (row['bbox'][1] + row['bbox'][3])/h]

  #Getting YOLO model predictions for this image id
  box2 = df2[df2.image_id == x['id']]
  if box2 is None:
    print("No Predictions")
  # print("Number of predictions from YOLO {}".format(str(len(test2))))
  box2['bbox_edit'] =  np.empty((len(box2), 0)).tolist()
  for idx, row in box2.iterrows():
    box2['bbox_edit'][idx] =  [row['bbox'][0]/w, 
                                row['bbox'][1]/h, 
                                (row['bbox'][0] + row['bbox'][2])/w, 
                                (row['bbox'][1] + row['bbox'][3])/h]


  #Getting YOLO model predictions for this image id
  box3 = df3[df3.image_id == x['id']]
  if box3 is None:
    print("No Predictions")
  # print("Number of predictions from YOLO {}".format(str(len(test2))))
  box3['bbox_edit'] =  np.empty((len(box3), 0)).tolist()
  for idx, row in box3.iterrows():
    box3['bbox_edit'][idx] =  [row['bbox'][0]/w, 
                                row['bbox'][1]/h, 
                                (row['bbox'][0] + row['bbox'][2])/w, 
                                (row['bbox'][1] + row['bbox'][3])/h]

  #Getting YOLO model predictions for this image id
  box4 = df4[df4.image_id == x['id']]
  if box4 is None:
    print("No Predictions")
  # print("Number of predictions from YOLO {}".format(str(len(test2))))
  box4['bbox_edit'] =  np.empty((len(box4), 0)).tolist()
  for idx, row in box4.iterrows():
    box4['bbox_edit'][idx] =  [row['bbox'][0]/w, 
                                row['bbox'][1]/h, 
                                (row['bbox'][0] + row['bbox'][2])/w, 
                                (row['bbox'][1] + row['bbox'][3])/h]

  # #Getting YOLO model predictions for this image id
  box5 = df5[df5.image_id == x['id']]
  if box5 is None:
    print("No Predictions")
  # print("Number of predictions from YOLO {}".format(str(len(test2))))
  box5['bbox_edit'] =  np.empty((len(box5), 0)).tolist()
  for idx, row in box5.iterrows():
    box5['bbox_edit'][idx] =  [row['bbox'][0]/w, 
                                row['bbox'][1]/h, 
                                (row['bbox'][0] + row['bbox'][2])/w, 
                                (row['bbox'][1] + row['bbox'][3])/h]
                          
  #Collate boxes, scores, lists
  image_boxes = [box1.bbox_edit.tolist(), 
                 box2.bbox_edit.tolist(), 
                 box3.bbox_edit.tolist(), 
                 box4.bbox_edit.tolist(),
                 box5.bbox_edit.tolist()]

  image_scores = [df1[df1.image_id == x['id']].score.tolist() , 
                  df2[df2.image_id == x['id']].score.tolist(), 
                  df3[df3.image_id == x['id']].score.tolist(),
                  df4[df4.image_id == x['id']].score.tolist(),
                  df5[df5.image_id == x['id']].score.tolist()]

  image_labels = [df1[df1.image_id == x['id']].category_id.tolist() , 
                  df2[df2.image_id == x['id']].category_id.tolist(),
                  df3[df3.image_id == x['id']].category_id.tolist(),
                  df4[df4.image_id == x['id']].category_id.tolist(),
                  df5[df5.image_id == x['id']].category_id.tolist()]
                

  boxes, scores, labels = wbf_mult_models(image_boxes, image_scores, image_labels, weights = weights, iou_thr=0.55)
  
  if boxes.shape[0] > 0:
      # x1,y1,x2,y2 -> x1,y1,w,h
      boxes[:, 2] -= boxes[:, 0]
      boxes[:, 3] -= boxes[:, 1]
      boxes[:, 0] *= w 
      boxes[:, 1] *= h
      boxes[:, 2] *= w
      boxes[:, 3] *= h

      bbox_score = scores
      for boxes_id in range(boxes.shape[0]):
          score = float(bbox_score[boxes_id])
          label = int(labels[boxes_id])
          box = boxes[boxes_id, :]
          
          image_result = {
              'image_id': x['id'],
              'category_id': label,
              'score': float(score),
              'bbox': box.tolist(),
          }

          results.append(image_result)


100%|██████████| 1000/1000 [07:01<00:00,  2.37it/s]


In [ ]:
filepath = f'json_outputs/ensembles/ensemble_bbox_results_{dataset_name}.json'
if os.path.exists(filepath):
    os.remove(filepath)
json.dump(results, open(filepath, 'w'), indent=4)

# Evaluation

In [ ]:
# Get evaluation score against validation set
val_annotations = os.path.join('data/val.json' )
coco_gt = COCO(val_annotations)
coco_dt = coco_gt.loadRes('val_json/ensembles/ensemble_bbox_results_val.json')
cocoEval = COCOeval(cocoGt=coco_gt, cocoDt=coco_dt, iouType='bbox')
cocoEval.evaluate()
cocoEval.accumulate()
cocoEval.summarize()

loading annotations into memory...
Done (t=0.53s)
creating index...
index created!
Loading and preparing results...
DONE (t=1.84s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=6.14s).
Accumulating evaluation results...
DONE (t=1.88s).
 Average Precision  (AP) @[ IoU=0.20:0.50 | area=   all | maxDets=100 ] = 0.728
 Average Precision  (AP) @[ IoU=0.20      | area=   all | maxDets=100 ] = 0.742
 Average Precision  (AP) @[ IoU=0.30      | area=   all | maxDets=100 ] = 0.737
 Average Precision  (AP) @[ IoU=0.40      | area=   all | maxDets=100 ] = 0.722
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.707


## Evaluating for each class

In [ ]:
for i in range(1,6):
  cocoEval.params.catIds = [i]
  cocoEval.evaluate()
  cocoEval.accumulate()
  cocoEval.summarize()

Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=1.05s).
Accumulating evaluation results...
DONE (t=0.35s).
 Average Precision  (AP) @[ IoU=0.20:0.50 | area=   all | maxDets=100 ] = 0.653
 Average Precision  (AP) @[ IoU=0.20      | area=   all | maxDets=100 ] = 0.688
 Average Precision  (AP) @[ IoU=0.30      | area=   all | maxDets=100 ] = 0.680
 Average Precision  (AP) @[ IoU=0.40      | area=   all | maxDets=100 ] = 0.635
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.598
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=1.95s).
Accumulating evaluation results...
DONE (t=1.44s).
 Average Precision  (AP) @[ IoU=0.20:0.50 | area=   all | maxDets=100 ] = 0.536
 Average Precision  (AP) @[ IoU=0.20      | area=   all | maxDets=100 ] = 0.554
 Average Precision  (AP) @[ IoU=0.30      | area=   all | maxDets=100 ] = 0.548
 Average Precision  (AP) @[ IoU=0.40      | area=   all | maxDets=100 ] = 0.527
 Average Precision